<a href="https://colab.research.google.com/github/alexandergribenchenko/Test_R5_DE/blob/main/Solucion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ¿Como crear una prueba de concepto de un Datawarehouse en el contexto del retail?
**Data Enginner Challenge - Grupo R5  (Presentado por: Alexander Ortega, Julio 2023)**

Este notebook presenta la solución al reto propuesto por el Grupo R5 para optar al rol de Data Enginner. El reto consiste en la implementacion de un datawarehouse en el conxtexto de negocio de ventas en tiendas fisicas de retail.

El notebook está compuesto de 3 secciones que dan respuesta a cada uno de los ítems descritos en la consigna compartida:

- **Sección 01. Diseño del datawarehouse:** Se presenta el diagrama del DWH implementado y se justifican la metodología implementada para su desarrollo.
- **Sección 02. Pipeline para la creación del Datawarehouse:** Se crea un pipeline que permite
- **Sección 03. Respuesta a preguntas particulares de negocio:** A partir del DWH implementado se responden a 2 preguntas especificas de negocio.

# Sección 01. Diseño del Datawarehouse

## S.01. Requerimiento

Diseñar un DWH con base en el dataset, se puede usar cualquier metodología.    
- Entregable: Gráfico representando el diseño, explicación de por qué se eligió la metodología teniendo en cuenta la escalabilidad y facilidad de uso para usuarios finales.

## S.01. Solución

### S.01.01. Gráfico con el diseño del Datawarehouse

![Texto alternativo](https://raw.githubusercontent.com/alexandergribenchenko/Test_R5_DE/main/images/DWH_R5_DE_Test.png)


### S.01.02. Justificación metodología

El diseño del Datawarehouse se generó bajo la metodología conocida como enfoque estrella (star schema).

En el enfoque estrella, se utiliza una tabla de hechos central que contiene las métricas o medidas clave que se desean analizar, como por ejemplo el total de venta de las órdenes. Esta tabla de hechos se conecta directamente con las tablas dimensionales a través de claves foráneas. Cada tabla dimensional representa una dimensión clave en el contexto de los datos, como el cliente, la tienda y la fecha.

En relación a la escalabilidad y a la facilidad del uso para usuarios finales podemos destacar los siguientes aspectos:


#### **Escalabilidad**

- **Agregación eficiente:** El enfoque estrella permite realizar agregaciones precalculadas y simplifica el cálculo de medidas en consultas analíticas. Esto mejora el rendimiento de las consultas y permite escalar el Data Warehouse para manejar grandes volúmenes de datos y consultas complejas de manera eficiente.
- **Incorporación de nuevas dimensiones:** El diseño del enfoque estrella facilita la incorporación de nuevas dimensiones en el futuro sin afectar la estructura existente. Esto brinda flexibilidad y escalabilidad para adaptarse a los cambios en los requisitos de análisis y las necesidades empresariales a medida que evolucionan con el tiempo.

#### **Facilidad de uso para usuarios finales**

- **Modelado intuitivo:** El enfoque estrella utiliza una estructura simple y fácil de comprender, lo que facilita a los usuarios finales comprender la relación entre las dimensiones y las medidas. Esto permite a los usuarios explorar y analizar los datos de manera más intuitiva, sin requerir un conocimiento profundo del modelo subyacente.
- **Consultas sencillas:** Las consultas en el enfoque estrella suelen ser más sencillas y directas, ya que se centran en las dimensiones clave y las medidas de interés. Los usuarios pueden formular consultas basadas en los atributos clave, como cliente, tienda o fecha, lo que facilita el análisis multidimensional y la obtención de información relevante rápidamente.

# Sección 02. Pipeline para la creación del Datawarehouse.

In [ ]:
import pandas as pd
import sqlite3
import os

In [27]:
path_github = 'https://raw.githubusercontent.com/alexandergribenchenko/Test_R5_DE/main/data/dataset.csv'

In [30]:
df = pd.read_csv(path_github, dtype=object)
df

,num_documento_cliente,tipo_documento_cliente,codigo_tienda,total_compra,tipo_tienda,latitud_tienda,longitud_tienda,id_barrio,nombre_barrio,fecha_compra
0,6861792605025121524,1,14812028,22218042,Tienda especializada,3.40211808,-76.52568134,737998244,Canaverales Los Samanes,2022-06-15 15:49:11
1,1552970090928921643,1,14388938,21583407,Tienda especializada,3.468765417,-76.48493586,737998896,Jorge Eliecer Gaitan,2022-10-02 22:14:54
2,-4075904842556875666,1,12370698,6185349,Tienda especializada,3.40074292,-76.52483946,737998244,Canaverales Los Samanes,2022-01-10 0:27:57
3,3539339627033221084,1,10313583,10313583,Tienda especializada,3.369543673,-76.52366403,738000400,Lili,2022-09-02 16:30:24
4,5595554501572542610,1,14358071,28716142,Tienda especializada,3.419045741,-76.49772993,737996734,El Poblado,2022-03-20 2:56:49
...,...,...,...,...,...,...,...,...,...,...
94605,-6859292209449934659,2,1116,558,Supermercado,3.47740375,-76.52670515,737998252,Chipichape,2022-04-30 14:38:39
94606,-1762474661445594290,2,745,372.5,Tienda Outlet,3.41432528,-76.54798624,737998095,Unidad Residencial Santiago de Cali,2022-01-07 23:41:02
94607,-1762474661445594290,2,745,372.5,Supermercado,3.45120421,-76.5329424,737998832,San Pedro,2022-11-09 13:46:59
94608,-2708241712915270749,2,4090,2045,Supermercado,3.42755068,-76.49999127,737995769,Julio Rincón,2022-06-15 2:55:38


In [37]:
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)

,name
0,DimTienda
1,DimBarrio
2,DimCliente
3,DimFecha
4,FactOrden


In [39]:
df_DimCliente.to_sql('DimCliente', conn, if_exists='replace', index=False)

11541

In [41]:
pd.read_sql_query("""
SELECT *
FROM DimCliente
""", conn)

,id_cliente,tipo_documento_cliente
0,-100445302626552670,1
1,-1004769336747862315,1
2,-1006371309257548646,1
3,-1008226411248037048,1
4,-1009950503667478931,1
...,...,...
11536,988313494016465029,1
11537,988313494016465029,2
11538,989380389770427585,1
11539,98946898638335839,1


In [38]:
df_DimCliente  =  df[['num_documento_cliente', 'tipo_documento_cliente']].drop_duplicates().\
                  sort_values(by='num_documento_cliente', ascending=True).\
                  rename(columns={'num_documento_cliente': 'id_cliente'}).\
                  reset_index(drop=True)

df_DimCliente

,id_cliente,tipo_documento_cliente
0,-100445302626552670,1
1,-1004769336747862315,1
2,-1006371309257548646,1
3,-1008226411248037048,1
4,-1009950503667478931,1
...,...,...
11536,988313494016465029,1
11537,988313494016465029,2
11538,989380389770427585,1
11539,98946898638335839,1


In [34]:
df_dim_cliente =  df[['tipo_tienda']].drop_duplicates().\
                  sort_values(by='tipo_tienda', ascending=True).\
                  reset_index(drop=True)

df_dim_cliente

,tipo_tienda
0,Hipermercado
1,Minimercado
2,Supermercado
3,Tienda Outlet
4,Tienda Regional
5,Tienda especializada


In [4]:
# Save the headlines DataFrame to a CSV file
data_path = './data'
df.to_csv(f'{create_folder(data_path)}/sample_file.csv', index=False)

In [9]:
conn = sqlite3.connect('sample_database.db')

In [10]:
c = conn.cursor()

In [11]:
dict_query = {}

dict_query['query_DimBarrio'] = """
CREATE TABLE IF NOT EXISTS "DimBarrio" (
  "id_barrio" int,
  "nombre_barrio" varchar,
  PRIMARY KEY ("id_barrio")
)
"""

dict_query['query_DimTienda'] = """
CREATE TABLE IF NOT EXISTS "DimTienda" (
  "id_tienda" int,
  "tipo_tienda" varchar,
  "latitud_tienda" float,
  "longitud_tienda" float,
  "id_barrio" int,
  PRIMARY KEY ("id_tienda"),
  CONSTRAINT "FK_DimTienda.id_barrio"
    FOREIGN KEY ("id_barrio")
      REFERENCES "DimBarrio"("id_barrio")
)
"""

dict_query['query_DimCliente'] = """
CREATE TABLE IF NOT EXISTS "DimCliente" (
  "id_cliente" int,
  "tipo_documento" int,
  PRIMARY KEY ("id_cliente")
)
"""

dict_query['query_DimFecha'] = """
CREATE TABLE IF NOT EXISTS "DimFecha" (
  "fecha_compra" datetime,
  PRIMARY KEY ("fecha_compra")
)
"""

dict_query['query_FactOrden'] = """
CREATE TABLE IF NOT EXISTS "FactOrden" (
  "id_orden" int,
  "total_compra" float,
  "fecha_compra" datetime,
  "id_cliente" int,
  "id_tienda" int,
  PRIMARY KEY ("id_orden"),
  CONSTRAINT "FK_FactOrden.fecha_compra"
    FOREIGN KEY ("fecha_compra")
      REFERENCES "DimFecha"("fecha_compra"),
  CONSTRAINT "FK_FactOrden.id_tienda"
    FOREIGN KEY ("id_tienda")
      REFERENCES "DimTienda"("id_tienda"),
  CONSTRAINT "FK_FactOrden.id_cliente"
    FOREIGN KEY ("id_cliente")
      REFERENCES "DimCliente"("id_cliente")
)
"""

In [20]:
for query in dict_query.keys():
  c.execute(dict_query[query])

In [21]:
list(dict_query.keys())

['query_DimBarrio',
 'query_DimTienda',
 'query_DimCliente',
 'query_DimFecha',
 'query_FactOrden']

In [24]:
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)

,name
0,DimTienda
1,DimBarrio
2,DimCliente
3,DimFecha
4,FactOrden


In [25]:
pd.read_sql_query("SELECT * FROM FactOrden", conn)

,id_orden,total_compra,fecha_compra,id_cliente,id_tienda


In [ ]:
df_

In [ ]:
df.to_sql('headlines', conn, if_exists='replace', index=False)

In [ ]:
headlines_df.to_sql('headlines', conn, if_exists='replace', index=False)

In [ ]:
conn.commit()

In [ ]:
df.to_sql('headlines', conn, if_exists='replace', index=False)

In [ ]:
conn.close()

In [ ]:
conn = sqlite3.connect('sample_database.db')

In [ ]:
df_leido = pd.read_sql_query("SELECT * FROM FactOrden", conn)
df_leido.head()

In [ ]:
df_leido = pd.read_sql_query("SELECT * FROM DimTienda", conn)
df_leido.head()

In [ ]:
CREATE TABLE DimBarrio (
  id_barrio int,
  nombre_barrio varchar,
  PRIMARY KEY (id_barrio)
);

CREATE TABLE DimTienda (
  id_tienda int,
  tipo_tienda varchar,
  latitud_tienda float,
  longitud_tienda float,
  id_barrio int,
  PRIMARY KEY (id_tienda),
  CONSTRAINT FK_DimTienda.id_barrio
    FOREIGN KEY (id_barrio)
      REFERENCES DimBarrio(id_barrio)
);

CREATE TABLE DimCliente (
  id_cliente int,
  tipo_documento int,
  PRIMARY KEY (id_cliente)
);

CREATE TABLE DimFecha (
  fecha_compra datetime,
  PRIMARY KEY (fecha_compra)
);

CREATE TABLE FactOrden (
  id_orden int,
  total_compra float,
  fecha_compra datetime,
  id_cliente int,
  id_tienda int,
  PRIMARY KEY (id_orden),
  CONSTRAINT FK_FactOrden.fecha_compra
    FOREIGN KEY (fecha_compra)
      REFERENCES DimFecha(fecha_compra),
  CONSTRAINT FK_FactOrden.id_tienda
    FOREIGN KEY (id_tienda)
      REFERENCES DimTienda(id_tienda),
  CONSTRAINT FK_FactOrden.id_cliente
    FOREIGN KEY (id_cliente)
      REFERENCES DimCliente(id_cliente)
);


In [ ]:
def create_folder(folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    return folder_name

In [ ]:
dict_input = { 'col_01': [1,2,3,4],
               'col_02': ['gato','perro','gato','perro'],
               'col_03': [1,5,6,7]
             }

df = pd.DataFrame.from_dict(dict_input)
df

,col_01,col_02,col_03
0,1,gato,1
1,2,perro,5
2,3,gato,6
3,4,perro,7
